In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd

from zipfile import ZipFile

In [2]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")

# add extension jpg to the file
def addextension(nm):
    return nm+".jpg"

train_df["Image_ID"] = train_df["Image_ID"].apply(addextension)
test_df["Image_ID"] = test_df["Image_ID"].apply(addextension)

print(train_df.head())

# extract zip file
with ZipFile("Train_Images.zip","r") as zip:
    zip.extractall()

with ZipFile("Test_Images.zip","r") as zip:
    zip.extractall()

          Image_ID            class   xmin   ymin  width  height
0  ID_007FAIEI.jpg  fruit_woodiness   87.0   87.5  228.0   311.0
1  ID_00G8K1V3.jpg  fruit_brownspot   97.5   17.5  245.0   354.5
2  ID_00WROUT9.jpg  fruit_brownspot  156.5  209.5  248.0   302.5
3  ID_00ZJEEK3.jpg    fruit_healthy  125.0  193.0  254.5   217.0
4  ID_018UIENR.jpg  fruit_brownspot   79.5  232.5  233.5   182.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    horizontal_flip = True,
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 3125 validated image filenames belonging to 3 classes.
Found 781 validated image filenames belonging to 3 classes.


### Build the model (Transfer VGG19 - Fine Tuning)

In [4]:
rescale = tf.keras.applications.vgg19.preprocess_input
base_model = tf.keras.applications.VGG19(input_shape=(256,256,3),include_top=False,weights="imagenet")
base_model.trainable = True

for layer in base_model.layers:
    if layer.name == "block3_pool":
        break
    layer.trainable = False

class Transfer_VGG19():
    def model(self,input):
        self.x = rescale(input)
        self.x = base_model(self.x,training=False)
        self.x = tf.keras.layers.GlobalAveragePooling2D()(self.x)
        self.x = tf.keras.layers.Flatten()(self.x)
        self.x = tf.keras.layers.Dense(128,activation="relu")(self.x)
        self.x = tf.keras.layers.Dropout(0.2,seed=42)(self.x)
        self.x = tf.keras.layers.Dense(64,activation="relu")(self.x)
        self.x = tf.keras.layers.Dropout(0.2,seed=42)(self.x)
        self.output = tf.keras.layers.Dense(3,activation="softmax")(self.x)
        self.model = tf.keras.Model(input,self.output,name="Transfer_VGG19")
        return self.model

TFVGG19 = Transfer_VGG19()
model = TFVGG19.model(tf.keras.Input(shape=(256,256,3)))
model.summary()
model.compile(tf.keras.optimizers.RMSprop(1e-5),tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])

Model: "Transfer_VGG19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 256, 256, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 256, 256, 3)      0         
                                                                 
 vgg19 (Functional)          (None, 8, 8, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 512)            

In [5]:
if __name__=="__main__":
    checkpoint = tf.keras.callbacks.ModelCheckpoint("makerere.h5",save_weights_only=False,save_best_only=True)
    model.fit(train_ds,epochs=10,validation_data=val_ds,callbacks=[checkpoint])
    best = tf.keras.models.load_model("makerere.h5")
    val_loss,val_acc = best.evaluate(val_ds)
    print("\nVal Accuracy: {:.2f} %".format(100*val_acc))
    print("Val Loss: {:.2f} %".format(100*val_loss))

Epoch 1/10
98/98 [==============================] - 98s 837ms/step - loss: 0.6550 - accuracy: 0.7469 - val_loss: 0.2431 - val_accuracy: 0.9219
Epoch 2/10
98/98 [==============================] - 69s 700ms/step - loss: 0.2533 - accuracy: 0.9098 - val_loss: 0.1204 - val_accuracy: 0.9641
Epoch 3/10
98/98 [==============================] - 69s 703ms/step - loss: 0.1425 - accuracy: 0.9514 - val_loss: 0.1117 - val_accuracy: 0.9654
Epoch 4/10
98/98 [==============================] - 69s 705ms/step - loss: 0.0950 - accuracy: 0.9661 - val_loss: 0.0826 - val_accuracy: 0.9718
Epoch 5/10
98/98 [==============================] - 70s 708ms/step - loss: 0.0781 - accuracy: 0.9773 - val_loss: 0.0528 - val_accuracy: 0.9795
Epoch 6/10
98/98 [==============================] - 70s 714ms/step - loss: 0.0532 - accuracy: 0.9818 - val_loss: 0.0578 - val_accuracy: 0.9808
Epoch 7/10
98/98 [==============================] - 70s 708ms/step - loss: 0.0475 - accuracy: 0.9850 - val_loss: 0.0605 - val_accuracy: 0.9834